In [10]:
from datasets import load_multitask_data, SentenceClassificationDataset, SentencePairDataset
from config import PretrainedConfig
import torch

In [2]:
sentiment_data, num_labels, paraphrase_data, similarity_data = load_multitask_data(sentiment_filename='data/ids-sst-train.csv',paraphrase_filename='data/quora-train.csv',similarity_filename='data/sts-train.csv')

Loaded 8544 train examples from data/ids-sst-train.csv
Loaded 141498 train examples from data/quora-train.csv
Loaded 6040 train examples from data/sts-train.csv


In [3]:
class tempArgs():
    def __init__(self,):
        self.local_files_only=True


In [4]:
tempargs = tempArgs()

In [5]:
para_train_data = SentencePairDataset(paraphrase_data,tempargs,isRegression=False)

In [7]:
batched_para_data = para_train_data.collate_fn(para_train_data.dataset)

In [48]:
import torch

sep_token_id = para_train_data.tokenizer.vocab['[SEP]']
# Determine the length needed for padding
max_length = 600
def concatenate_with_padding_and_mask(tensor1, tensor2,mask1,mask2):
    concatenated_tensors = []

    
    

    for i in range(len(tensor1)):
        non_zero_indices1 = torch.nonzero(mask1[i], as_tuple=False).squeeze()
        non_zero_indices2 = torch.nonzero(mask2[i], as_tuple=False).squeeze()
        
        # Get the maximum non-zero index for each tensor separately
        max_index1 = torch.max(non_zero_indices1).item() + 1
        max_index2 = torch.max(non_zero_indices2).item() + 1

        # Concatenate the tensors and update attention masks
        # the second tensor concatenatesfrom position 1 because the separator 
        concatenated_tensor = torch.cat((tensor1[i][:max_index1], tensor2[i][1:max_index2]), dim=0)

        # Pad the concatenated tensor and mask to the maximum length
        padding_length = max_length - len(concatenated_tensor)
        padding = torch.zeros(padding_length)

        concatenated_tensor = torch.cat((concatenated_tensor, padding), dim=0)

        concatenated_tensors.append(concatenated_tensor)

    return torch.stack(concatenated_tensors)

# Example usage with batch_size=2 and sequence_length=6
tensor1 = torch.tensor([[1, 2, 3, 0, 0, 0],
                        [7, 8, 0, 0, 0, 0]])
tensor2 = torch.tensor([[0, 0, 0, 4, 5, 6],
                        [9, 0, 0, 0, 0, 0]])


mask1 = torch.tensor([[1, 1, 1, 0, 0, 0],
                      [1, 1, 0, 0, 0, 0]])
mask2 = torch.tensor([[0, 0, 0, 1, 1, 1],
                      [1, 0, 0, 0, 0, 0]])


result= concatenate_with_padding_and_mask(tensor1, tensor2, mask1, mask2)
print(result.shape)
print("Concatenated tensor:", result)



torch.Size([2, 600])
Concatenated tensor: tensor([[1., 2., 3.,  ..., 0., 0., 0.],
        [7., 8., 0.,  ..., 0., 0., 0.]])


In [49]:
concatenation = concatenate_with_padding_and_mask(batched_para_data['token_ids_1'],batched_para_data['token_ids_2'],batched_para_data['attention_mask_1'],batched_para_data['attention_mask_2'])


In [51]:
para_train_data.tokenizer.convert_ids_to_tokens(concatenation[100])

['[CLS]',
 'how',
 'can',
 'i',
 'specifically',
 'improve',
 'my',
 'english',
 '?',
 '[SEP]',
 'what',
 'should',
 'i',
 'do',
 'to',
 'improve',
 'my',
 'english',
 '?',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]